## Source code for _Investigating particle size-flux relationships and the biological pump across a range of plankton ecosystem states from coastal to oligotrophic_

Manuscript by: __Christian K. Fender, Thomas B. Kelly, Lionel Guidi, Mark D. Ohman, Matthew C. Smith, and Michael R. Stukel__

Code by: __Thomas B. Kelly and Christian K. Fender (FSU)__  

Manuscript submitted to Frontiers in Marine Science (March 2019)


### _Abstract_

Sinking particles transport organic carbon produced in the surface ocean to the ocean interior, leading to net storage of atmospheric CO2 in the deep ocean.  The rapid growth of in situ imaging technology has the potential to revolutionize our understanding of particle flux attenuation in the ocean; however, estimating particle flux from particle size and abundance (measured directly by in situ cameras) is challenging. Sinking rates are dependent on several factors, including particle excess density and porosity, which vary based on particle origin and type. Additionally, particle characteristics are transformed while sinking. We compare optically-measured particle size spectra profiles (Underwater Vision Profiler 5, UVP) with contemporaneous measurements of particle flux made using sediment traps and $^{234}Th:^{238}U$ disequilibrium on six process cruises from the California Current Ecosystem (CCE) LTER Program. These measurements allow us to assess the efficacy of using size-flux relationships to estimate fluxes from optical particle size measurements. We find that previously published parameterizations that estimate carbon flux from UVP profiles are a poor fit to direct flux measurements in the CCE.  This discrepancy is found to result primarily from the important role of fecal pellets in particle flux.  These pellets are primarily in a size range (i.e., 100 – 400 µm) that is not well-resolved as images by the UVP due to the resolution of the sensor.  We develop a new, CCE-optimized algorithm for estimating carbon flux from UVP data in the southern California Current (Flux = $\sum_{i=1}^{x}{n_i A d_i^B ∆d_i }$), with A = 13.45, B = 1.35, d = particle diameter (mm) and Flux in units of $mg C m^{-2} d^{-1}$.  We caution, however, that increased accuracy in flux estimates derived from optical instruments will require devices with greater resolution, the ability to differentiate fecal pellets from low porosity marine snow aggregates, and improved sampling of rapidly sinking fecal pellets.  We also find that the particle size-flux relationships may be different within the euphotic zone than in the shallow twilight zone and hypothesize that the changing nature of sinking particles with depth must be considered when investigating the remineralization length scale of sinking particles in the ocean.

[![DOI](https://zenodo.org/badge/178078252.svg)](https://zenodo.org/badge/latestdoi/178078252)


---

### Note on downloading the Raw Data

An account with Ecotaxa will be necessary for the following steps.

1. From the home page, select the drop down window next to Action at the top right. Select the Particle Module.
2. Enter the relevant filters. For this study all CCELTER Particle Projects were selected.
3. Select Export Selection.
4. Download the RAW data format. Only this format will work with the provided code.
5. From the downloaded files, only the PAR files and the metadata summary file were used for analysis.

---

### Start by loading the source files

In [ ]:
source('source.uvp.r') ## Functions to make uvp data objects
source('source.uvp.add.r') ## functions to add values to uvp data objects
source('source.r') ## General purpose functions (optional)

Next we will define directories and load the UVP metadata files

In [ ]:
#### Load the Data
input.dir = '../UVP Data/All PAR files/'
input.dir.meta = '../UVP Data/All Metadata files/'

raw.files = list.files(input.dir)
meta.files = list.files(input.dir.meta)

## Reading in only first metadata file.
meta = as.data.frame(fread(paste0(input.dir.meta, meta.files[1])))

## Combining with the rest.
for (i in 2:length(meta.files)) {
  meta = rbind(meta, as.data.frame(fread(paste0(input.dir.meta, meta.files[i]))))
}

_Optional:_ Here we load up the CTD and Sed Trap datasets and save them as rdata files for easy loading later.

In [ ]:
#### CTD and Sed Trap File section
#ctd.file = '../Other Data/CTD Downcast Data.xlsx'
#ctd = read.xlsx(ctd.file)
#ctd$Time = as.POSIXct(ctd$Datetime.GMT, tz = 'UTC') ## Get timestamps
#save(ctd, file = '../Other Data/CTD.rdata')

#sedtrap = read.xlsx('../Other Data/Sediment Trap.xlsx')
#sedtrap$Deployment.Time = as.POSIXct(sedtrap$Deployment.Time)
#sedtrap$Recovery.Datetime = as.POSIXct(sedtrap$Recovery.Datetime)
#save(sedtrap, file = '../Other Data/SedTrap.rdata')

In [ ]:
## Load saved datasets
load('../Other Data/CTD.rdata')
load('../Other Data/SedTrap.rdata')

### Build a UVP data object
First we build one to make sure the system is working, and then we'll loop it for all the uvp cast files.

In [ ]:
##Initialize empty data structure, set Files
MasterList = init.raw(Dir = input.dir, File = raw.files[269]) # Picked a random file.
MasterList$params$dz = 5
MasterList$params$A = 13.7
MasterList$params$b = 1.55

In [ ]:
## Load our data: set data.raw, fill in meta
MasterList = load.raw(MasterList, meta, TRUE)
MasterList = filter.by.size(MasterList, d.min = 0.102, d.max = 1.5) # Filtering to 2nd smallest size bin

Setup bins based on a logrithmic scale.

In [ ]:
k = 3 ## Our current bins (increase for higher resolution, Guidi = 3)
bins = c(2^(c((5*k):(11*k))/k))

bins = floor(bins)
delta.bins = diff(bins)

## Save bin values
MasterList$params$bins = bins
MasterList$params$delta.bins = delta.bins
MasterList$params$n = length(bins)

In [ ]:
## Make the bins
MasterList = make.bins(MasterList)

## Calculate the flux (5m bins)
MasterList = calc.flux(MasterList)

## Add export values (sed trap data)
MasterList = add.export(MasterList, thorium = thorium, sedtrap = sedtrap)

##Add ctd metadata
MasterList = add.ctd(MasterList, ctd, verbose = TRUE)

### Done with test

---

### Full scale
If that all works, then we can make a function to loop over all the files and make a list of each uvp cast object:

In [ ]:
## See source.uvp.r for code on run.all.data(). This will need to be edited and provided as a template only.
cruises = run.all.data(raw.files, A = 13.45, b = 1.35, dz = 5, ctd = ctd, verbose = FALSE, k = 3)

### Save result

In [ ]:
save(cruises, file = '../rstates/cruises.rdata')